Eval modele
solution d'appel du retriver sur une query
sur la base des attendu


In [1]:
from pipeline import rag_pipeline

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import json
import itertools

# Chemin vers le fichier JSON
chemin_queries = 'dataset_eval/queries.json'
chemin_answers = 'dataset_eval/answers.json'

try:
    # Ouvrir et lire le fichier JSON
    with open(chemin_queries, 'r', encoding='utf-8') as f:
        # Charger le contenu JSON dans un dictionnaire Python
        queries = json.load(f)

except FileNotFoundError:
    print(f"Erreur : Le fichier '{chemin_fichier}' n'a pas été trouvé.")
except json.JSONDecodeError:
    print(f"Erreur : Le contenu de '{chemin_fichier}' n'est pas un JSON valide.")
except Exception as e:
    print(f"Une erreur inattendue est survenue : {e}")

try:
    # Ouvrir et lire le fichier JSON
    with open(chemin_answers, 'r', encoding='utf-8') as f:
        # Charger le contenu JSON dans un dictionnaire Python
        answers = json.load(f)

except FileNotFoundError:
    print(f"Erreur : Le fichier '{chemin_fichier}' n'a pas été trouvé.")
except json.JSONDecodeError:
    print(f"Erreur : Le contenu de '{chemin_fichier}' n'est pas un JSON valide.")
except Exception as e:
    print(f"Une erreur inattendue est survenue : {e}")

first_five_values = itertools.islice(queries.items(), 5)

for key, query in first_five_values:
    print(query, ":", answers[key])
    print("zzzzzzz")
    print(rag_pipeline(query))
    print("zzzzzzz")
    print(answers[key])
    print("")
    print("*****************")
